In [47]:
import edward as ed
import tensorflow as tf
from edward.models import Multinomial,Bernoulli,Binomial, Normal
from pandas_plink import read_plink
import numpy as np
import pandas as pd
from collections import OrderedDict as odict

In [48]:
N = 5000
M = 100000
K = 25


In [49]:
def snp_neural_network(z,w):
    z_tile = tf.tile(tf.reshape(z, [N, 1, K]), [1, M, 1])
    w_tile = tf.tile(tf.reshape(w, [1, M, K]), [N, 1, 1])
    h = tf.concat([z_tile, w_tile], 2)
    h = tf.layers.dense(h, 512, activation=tf.nn.relu)
    h = tf.layers.dense(h, 512, activation=tf.nn.relu)
    h = tf.layers.dense(h, 1, activation=None)
    return tf.reshape(h, [N, M])

In [50]:
def trait_neural_network(z, x):
    eps = Normal(loc=0.0, scale=1.0, sample_shape=[N, 1])
    h = tf.concat([z, x, eps], 1)
    h = tf.layers.dense(h, 32, activation=tf.nn.relu)
    h = tf.layers.dense(h, 256, activation=tf.nn.relu)
    h = tf.concat([z, h], 1) # include connection to z for output layer
    h = tf.layers.dense(h, 1, activation=None)
    return tf.reshape(h, [N])

In [51]:
z = Normal(loc=0.0, scale=1.0, sample_shape=[N, K])
w = Normal(loc=0.0, scale=1.0, sample_shape=[M, K])
logits = snp_neural_network(z, w)
x = Multinomial(total_count=2.0, logits=logits)
y = Bernoulli(logits=trait_neural_network(z, x))

In [52]:
z.shape

TensorShape([Dimension(5000), Dimension(25)])

In [53]:
w.shape

TensorShape([Dimension(100000), Dimension(25)])

In [54]:
x.shape

TensorShape([Dimension(5000), Dimension(100000)])

In [55]:
y.shape


TensorShape([Dimension(5000)])

In [15]:
f_name = '/Users/raouldias/Desktop/Extend/extend_csp_data_annon'
pheno_name = '/Users/raouldias/Desktop/Extend/extend_phenotype.txt'

In [16]:
(bim,fam,bed) = read_plink(f_name)

Mapping files: 100%|██████████| 3/3 [00:01<00:00,  1.57it/s]


In [17]:
bed = bed.rechunk((100,7367))

In [18]:
def read_pheno(f_name):
    header = odict(
        [
            ("FID", str),
            ("IID", str),
            ("AGE", str),
            ("BMI", str),
            ("T2D", str),
        ]
    )

    df = _read_csv(f_name, header)
    df["i"] = range(df.shape[0])
    return df
    
def _read_csv(fn, header):
    return pd.read_csv(
        fn,
        delim_whitespace=True,
        header=None,
        names=header.keys(),
        dtype=header,
        compression=None,
        engine="c",
    )

In [19]:
pheno_df = read_pheno(pheno_name)